In [1]:
import traceback
import sys
from functools import lru_cache
from web3 import Web3
from web3.auto import w3
from web3.contract import Contract
from web3._utils.events import get_event_data

from web3._utils.abi import exclude_indexed_event_inputs, get_abi_input_names, get_indexed_event_inputs, normalize_event_input_types
from web3.exceptions import MismatchedABI, LogTopicError
from web3.types import ABIEvent
from eth_utils import event_abi_to_log_topic, to_hex
from hexbytes import HexBytes

import json
import re

def decode_tuple(t, target_field):
    output = dict()
    for i in range(len(t)):
        if isinstance(t[i], (bytes, bytearray)):
            output[target_field[i]['name']] = to_hex(t[i])
        elif isinstance(t[i], (tuple)):
            output[target_field[i]['name']] = decode_tuple(t[i], target_field[i]['components'])
        else:
            output[target_field[i]['name']] = t[i]
    return output

def decode_list_tuple(l, target_field):
    output = l
    for i in range(len(l)):
        output[i] = decode_tuple(l[i], target_field)
    return output

def decode_list(l):
    output = l
    for i in range(len(l)):
        if isinstance(l[i], (bytes, bytearray)):
            output[i] = to_hex(l[i])
        else:
            output[i] = l[i]
    return output

def convert_to_hex(arg, target_schema):
    """
    utility function to convert byte codes into human readable and json serializable data structures
    """
    output = dict()
    for k in arg:
        if isinstance(arg[k], (bytes, bytearray)):
            output[k] = to_hex(arg[k])
        elif isinstance(arg[k], (list)) and len(arg[k]) > 0:
            target = [a for a in target_schema if 'name' in a and a['name'] == k][0]
            if target['type'] == 'tuple[]':
                target_field = target['components']
                output[k] = decode_list_tuple(arg[k], target_field)
            else:
                output[k] = decode_list(arg[k])
        elif isinstance(arg[k], (tuple)):
            target_field = [a['components'] for a in target_schema if 'name' in a and a['name'] == k][0]
            output[k] = decode_tuple(arg[k], target_field)
        else:
            output[k] = arg[k]
    return output

@lru_cache(maxsize=None)
def _get_contract(address, abi):
    """
    This helps speed up execution of decoding across a large dataset by caching the contract object
    It assumes that we are decoding a small set, on the order of thousands, of target smart contracts
    """
    if isinstance(abi, (str)):
        abi = json.loads(abi)

    contract = w3.eth.contract(address=Web3.toChecksumAddress(address), abi=abi)
    return (contract, abi)

def decode_tx(address, input_data, abi):
    if abi is not None:
        try:
            (contract, abi) = _get_contract(address, abi)
            func_obj, func_params = contract.decode_function_input(input_data)
            target_schema = [a['inputs'] for a in abi if 'name' in a and a['name'] == func_obj.fn_name][0]
            decoded_func_params = convert_to_hex(func_params, target_schema)
            return (func_obj.fn_name, json.dumps(decoded_func_params), json.dumps(target_schema))
        except:
            e = sys.exc_info()[0]
        return ('decode error', repr(e), None)
    else:
        return ('no matching abi', None, None)

In [2]:
import requests

def get_contract_abi(address, contract_json):
    
    address_obj = contract_json.get(address)
        
    if not address_obj or not address_obj.get('abi'):
        
        res = requests.get('https://api.etherscan.io/api?module=contract&action=getabi&address=%s' % address)
        
        res = res.json()
        
        if not address_obj:
            address_obj = {}

        
        # TODO: check for existing info from another functions        
        
        
        if res.get('status') == '1':
            address_obj = {'abi': res["result"]}
            contract_json[address] = address_obj
            
            return res['result']
        else:
            print('Failed to fetch ABI for %s' % address)

    else:
        return address_obj['abi']
    
def get_contract_decimals(address, contract_json):
    
    address_obj = contract_json.get(address)
        
    if not address_obj or not address_obj.get('decimals'):
        
        res = requests.get('https://api.ethplorer.io/getTokenInfo/%s?apiKey=freekey' % address)
        res = res.json()
        
        if not address_obj:
            address_obj = {}
        
        for k, v in res.items():
            
            address_obj[k] = v
            
        contract_json[address] = address_obj
        
        return int(res['decimals'])
    else:
        return int(address_obj['decimals'])
#     res.json()['decimals']

def get_contract_symbol(address, contract_json):
    
    address_obj = contract_json.get(address)
        
    if not address_obj or not address_obj.get('symbol'):
        
        res = requests.get('https://api.ethplorer.io/getTokenInfo/%s?apiKey=freekey' % address)
        res = res.json()
        
        if not address_obj:
            address_obj = {}
        
        for k, v in res.items():
            
            address_obj[k] = v
            
        contract_json[address] = address_obj
        
        return res['symbol']
    else:
        return address_obj['symbol']
    
def get_transactions_by_address(address):

    res = requests.get('https://api.ethplorer.io/getAddressTransactions/%s?apiKey=freekey&showZeroValues=true' % address)
    
    res = res.json()    
    
    return res

In [3]:
WALLET_ADDRESS = '0x0C4EcE75800273Bb516394b8a82B6B180b5f86c4'

In [4]:
with open('contract_info.json', 'r') as f:
    contract_info = json.load(f)

In [6]:
ankr_url = 'https://rpc.ankr.com/eth'

web3 = Web3(Web3.HTTPProvider(ankr_url))

print(web3.isConnected())

True


In [10]:
from web3._utils.events import get_event_data
from functools import lru_cache
import json
import traceback

@lru_cache(maxsize=None)
def _get_topic2abi(abi):
  if isinstance(abi, (str)):
    abi = json.loads(abi)

  event_abi = [a for a in abi if a['type'] == 'event']
  topic2abi = {event_abi_to_log_topic(_): _ for _ in event_abi}
  return topic2abi

@lru_cache(maxsize=None)
def _get_hex_topic(t):
  hex_t = HexBytes(t)
  return hex_t


def decode_log(data, topics, abi):
  if abi is not None:
    try:
      topic2abi = _get_topic2abi(abi)
      
      log = {
        'address': None, #Web3.toChecksumAddress(address),
        'blockHash': None, #HexBytes(blockHash),
        'blockNumber': None,
        'data': data, 
        'logIndex': None,
        'topics': [_get_hex_topic(_) for _ in topics],
        'transactionHash': None, #HexBytes(transactionHash),
        'transactionIndex': None
      }
      event_abi = topic2abi[log['topics'][0]]
      evt_name = event_abi['name']

      data = get_event_data(w3.codec, event_abi, log)['args']
      target_schema = event_abi['inputs']
      decoded_data = convert_to_hex(data, target_schema)
      
    
      return (evt_name, json.dumps(decoded_data), json.dumps(target_schema))
    except Exception:
      return ('decode error', traceback.format_exc(), None)
    
  else:
    return ('no matching abi', None, None)
  

In [54]:
def parse_transfer(transaction):
    
    transaction_hash = transaction['hash']

    print(transaction_hash)

    transaction = web3.eth.get_transaction(transaction_hash)
    input_ = transaction.input

    abi = get_contract_abi(transaction.to,  contract_info)
    output = decode_tx(transaction.to, input_, abi)
    print('function called: ', output[0])

    #     print(output)

    transaction_logs = web3.eth.get_transaction_receipt(transaction_hash)
    print(transaction_logs)

    token_contract = transaction_logs.to

    decimals = get_contract_decimals(token_contract, contract_info)

    symbol = get_contract_symbol(token_contract, contract_info)

    print(decimals, symbol)



    last_transaction = transaction_logs.logs[-1]

    pair_abi = '[{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"address","name":"spender","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"sender","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount0","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount1","type":"uint256"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"Burn","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"sender","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount0","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount1","type":"uint256"}],"name":"Mint","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"sender","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount0In","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount1In","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount0Out","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount1Out","type":"uint256"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"Swap","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint112","name":"reserve0","type":"uint112"},{"indexed":false,"internalType":"uint112","name":"reserve1","type":"uint112"}],"name":"Sync","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Transfer","type":"event"},{"constant":true,"inputs":[],"name":"DOMAIN_SEPARATOR","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"MINIMUM_LIQUIDITY","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"PERMIT_TYPEHASH","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"internalType":"address","name":"","type":"address"},{"internalType":"address","name":"","type":"address"}],"name":"allowance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"approve","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"balanceOf","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"to","type":"address"}],"name":"burn","outputs":[{"internalType":"uint256","name":"amount0","type":"uint256"},{"internalType":"uint256","name":"amount1","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"factory","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"getReserves","outputs":[{"internalType":"uint112","name":"_reserve0","type":"uint112"},{"internalType":"uint112","name":"_reserve1","type":"uint112"},{"internalType":"uint32","name":"_blockTimestampLast","type":"uint32"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"_token0","type":"address"},{"internalType":"address","name":"_token1","type":"address"}],"name":"initialize","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"kLast","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"to","type":"address"}],"name":"mint","outputs":[{"internalType":"uint256","name":"liquidity","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"name","outputs":[{"internalType":"string","name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"nonces","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"},{"internalType":"uint256","name":"deadline","type":"uint256"},{"internalType":"uint8","name":"v","type":"uint8"},{"internalType":"bytes32","name":"r","type":"bytes32"},{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"permit","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"price0CumulativeLast","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"price1CumulativeLast","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"to","type":"address"}],"name":"skim","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"uint256","name":"amount0Out","type":"uint256"},{"internalType":"uint256","name":"amount1Out","type":"uint256"},{"internalType":"address","name":"to","type":"address"},{"internalType":"bytes","name":"data","type":"bytes"}],"name":"swap","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"symbol","outputs":[{"internalType":"string","name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"sync","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"token0","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"token1","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"totalSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"transfer","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"from","type":"address"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"transferFrom","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"}]'

    output = decode_log(
        last_transaction['data'],
        last_transaction['topics'],
        pair_abi
    )

    args = json.loads(output[1])
    _from = args['from']
    _to = args['to']
    amount = args['value']

    amount = amount / (1 * 10 ** decimals)
    
    fee = transaction_logs['gasUsed'] * transaction_logs['effectiveGasPrice'] / (1 * 10 ** 18)

    print(_from, _to, amount, fee, 'ETH') 
    

0x30307e6560a077e4d5cefd867c0ffa3d6b02eaba3280c4595632e571ce1ddda2
function called:  transfer
AttributeDict({'blockHash': HexBytes('0xe484cb3d4821b278f752e955ca717acfb3912364edf1712cba2165d31fe42cc0'), 'blockNumber': 13466801, 'contractAddress': None, 'cumulativeGasUsed': 22158298, 'effectiveGasPrice': 67074448864, 'from': '0x0C4EcE75800273Bb516394b8a82B6B180b5f86c4', 'gasUsed': 46097, 'logs': [AttributeDict({'address': '0xdAC17F958D2ee523a2206206994597C13D831ec7', 'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'), HexBytes('0x0000000000000000000000000c4ece75800273bb516394b8a82b6b180b5f86c4'), HexBytes('0x000000000000000000000000f16c48077e70f133ce5be05ee4aa65a61bea8c1e')], 'data': '0x0000000000000000000000000000000000000000000000000000000023c34600', 'blockNumber': 13466801, 'transactionHash': HexBytes('0x30307e6560a077e4d5cefd867c0ffa3d6b02eaba3280c4595632e571ce1ddda2'), 'transactionIndex': 325, 'blockHash': HexBytes('0xe484cb3d4821b278f752e955ca

In [55]:
def parse_all_transactions(address):

    transactions = get_transactions_by_address(address)
    
    zero_value_transactions = [t for t in transactions if t['value'] == 0]
    
    for t in zero_value_transactions:
        
        if not t['success']:
            continue
        
        transaction_hash = t['hash']
        
        print('transaction hash', transaction_hash)
        print(t)
        
        transaction = web3.eth.get_transaction(transaction_hash)
        input_ = transaction.input

        abi = get_contract_abi(transaction.to,  contract_info)
        output = decode_tx(transaction.to, input_, abi)
        print('function called: ', output[0])
        
        if output[0] == 'transfer':
            
            parse_transfer(t)
        
        elif output[0] == 'swapExactTokensForTokens':
            
            print('arguments: ', json.dumps(json.loads(output[1]), indent=2))

            args = json.loads(output[1])

            decimals = get_contract_decimals(args["path"][0], contract_info)
            symbol = get_contract_symbol(args["path"][0], contract_info)
            amount_out = args["amountIn"] / (1 * 10 ** decimals)

            output_decimals = get_contract_decimals(args["path"][-1], contract_info)
            output_symbol = get_contract_symbol(args["path"][-1], contract_info)

            transaction_logs = web3.eth.get_transaction_receipt(transaction_hash)


            last_transaction = transaction_logs.logs[-1]

            output = decode_log(
                last_transaction['data'],
                last_transaction['topics'],
                pair_abi
            )

            print('event emitted: ', output[0])
            print('arguments: ', json.dumps(json.loads(output[1]), indent=2))
            print()

            args = json.loads(output[1])

            amount_in = args['amount0Out'] / (1 * 10 ** output_decimals)
            
            fee = transaction_logs['gasUsed'] * transaction_logs['effectiveGasPrice'] / (1 * 10 ** 18)

            print(amount_out, symbol, amount_in, output_symbol, fee, 'ETH')
        else:
            print('Not supported', output[0])
            

In [56]:
parse_all_transactions(WALLET_ADDRESS)

transaction hash 0x30307e6560a077e4d5cefd867c0ffa3d6b02eaba3280c4595632e571ce1ddda2
{'timestamp': 1634897135, 'from': '0x0c4ece75800273bb516394b8a82b6b180b5f86c4', 'to': '0xdac17f958d2ee523a2206206994597c13d831ec7', 'hash': '0x30307e6560a077e4d5cefd867c0ffa3d6b02eaba3280c4595632e571ce1ddda2', 'value': 0, 'input': '0xa9059cbb000000000000000000000000f16c48077e70f133ce5be05ee4aa65a61bea8c1e0000000000000000000000000000000000000000000000000000000023c34600', 'success': True}
function called:  transfer
0x30307e6560a077e4d5cefd867c0ffa3d6b02eaba3280c4595632e571ce1ddda2
function called:  transfer
AttributeDict({'blockHash': HexBytes('0xe484cb3d4821b278f752e955ca717acfb3912364edf1712cba2165d31fe42cc0'), 'blockNumber': 13466801, 'contractAddress': None, 'cumulativeGasUsed': 22158298, 'effectiveGasPrice': 67074448864, 'from': '0x0C4EcE75800273Bb516394b8a82B6B180b5f86c4', 'gasUsed': 46097, 'logs': [AttributeDict({'address': '0xdAC17F958D2ee523a2206206994597C13D831ec7', 'topics': [HexBytes('0xddf252

0 SLP
0x0C4EcE75800273Bb516394b8a82B6B180b5f86c4 0x75BB69D79a6d74B22417efA20e696D729657749F 2804.0 0.006885097827498865 ETH


In [26]:
transactions = get_transactions_by_address(WALLET_ADDRESS)


In [57]:
with open('contract_info.json', 'w') as f:
    json.dump(contract_info, f, indent=4)